### Base Vector Animation

This produces a visualization of how the SSP map with a single object located at $(0, 0)$, out of view, changes as a function of the base vectors for $X$ and $Y$. Specifically, these base vectors are exponentiated by continuously-varied quantities, which helps to visualize how the landscape changes continuously.

Technically, what this demonstrates is that $(X^a)^b \ne X^{ab}$ in general (since $a, b \in \mathbb{R}$), otherwise this animation would do nothing more than zoom in and out of a fixed landscape. Clearly something other than zooming is occurring.

In [ ]:
%matplotlib inline

In [ ]:
import base64

from IPython.display import HTML
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from nengo.utils.progress import Progress, ProgressTracker
import nengo_spa as spa

from ssp.pointers import BaseVectors

In [ ]:
d = 64
rng = np.random.RandomState(seed=0)
gen = BaseVectors(d, rng=rng)

In [ ]:
xs = np.linspace(0.05, 1, 300)  # normally -1 to 1
ys = np.linspace(0.05, 1, 300)  # normally -1 to 1

# cmap = sns.diverging_palette(150, 275, s=80, l=55, as_cmap=True)
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)

In [ ]:
X = spa.SemanticPointer(next(gen), name="X")
Y = spa.SemanticPointer(next(gen), name="Y")

n_frames = 100
n_frames_delay = 0
reverse_end = True
interval = 80  # milliseconds


def heatmap(sims):
    return plt.imshow(
        sims,
        interpolation='none',
        extent=(xs[0], xs[-1], ys[0], ys[-1]),
        #vmin=-1,
        #vmax=1,
        cmap=cmap,
        animated=True,
    )


fig = plt.figure(figsize=(6, 6))

ims = []
with ProgressTracker(
    True, Progress("Animating", "Animation", n_frames)
) as progress_bar:
    sims = np.zeros((len(ys), len(xs)))
    for frame in range(n_frames):
        a = 1
        b = (1 + 5**0.5) / 2  # golden ratio
        scale = 20
        k = frame/(n_frames - 1)

        # note: SSPs w/ HRR do *not* satisfy ((x**a)**b) == x**(a*b)
        # hence bX and bY are not shifting the axes
        bX = (X**(1 + a*k))
        bY = (Y**(1 + b*k))
        for i, x in enumerate(xs):
            for j, y in enumerate(ys):
                # the zero'th element is the dot-product with
                # the identity vector (X**0 * Y**0 regardless of X, Y)
                # this is equivalent to visualizing the similarity map
                # with one object located at (0, 0)
                sims[i, j] = (bX**(x*scale) * bY**(y*scale)).v[0]

        ims.append([heatmap(sims)])
        progress_bar.total_progress.step()

# reverse to first frame and then delay before repeating
if reverse_end:
    ims.extend(ims[::-1])
for _ in range(n_frames_delay):
    ims.append(ims[-1])

plt.xticks([])
plt.yticks([])

ani = animation.ArtistAnimation(fig, ims, interval=interval, blit=True)

# plt.show()

plt.close()

In [ ]:
# HTML(ani.to_html5_video())
# save it as a a gif and also embed it in the notebook as HTML

fname = ".temp.gif"
ani.save(fname, writer='imagemagick')
gif = open(fname, "rb").read()
gif_base64 = base64.b64encode(gif).decode()
HTML('<img src="data:image/gif;base64,{0}" />'.format(gif_base64))